# Meet the Course Applicants

**Goal:** To get a better sense of what kind of people sign up for Applied Data Science Lab — where they're from, how old are they, what have they previously studied, and more.

- Extract and transform applicant demographic information using PyMongo.
- Enrich demographic information using an open-source library.
- Create a choropleth map to visualize nationality.
- Build a sorting function to visualize education level.

In [7]:
from pprint import PrettyPrinter

import pandas as pd
import plotly.express as px
from country_converter import CountryConverter
from pymongo import MongoClient

Instantiate PrettyPrinter

In [9]:
pp = PrettyPrinter(indent=2)
print("pp type:", type(pp))

pp type: <class 'pprint.PrettyPrinter'>


Connect to MongoDB Client